In [119]:
import gym
import numpy as np
from random import choice

In [120]:
# Define the tic-tac-toe environment

class TicTacToeEnv(gym.Env):
    def __init__(self):
        # The game board is a 2-dimensional 3x3 array
        self.board = np.zeros((3, 3), dtype=int)
        # There are 9 discreet actions, corresponding to placing a token in one of the nine spaces
        self.action_space = gym.spaces.Discrete(9)
        # The game board as an observation_space
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(3, 3), dtype=int)
        self.current_player = 1
        self.winner = None
        self.done = False

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        self.winner = None
        self.done = False
        return self.board

    # Docs: https://www.gymlibrary.dev/api/core/
    def step(self, action):
        """Performs a single step in a RL Episode. Takes an action as an argument, returns tuple of
        format: (observation, reward, terminated, truncated, info)"""
        if self.done:
            return self.board, 0, True, {}

        # Translate action number into row & column
        row, col = divmod(action, 3)
        # If that box is unoccupied, place token. Else, give penalty to agent.
        if self.board[row][col] == 0:
            self.board[row][col] = self.current_player
        else:
            self.current_player = -self.current_player
            return self.board, -1, False, {}

        self.winner = self.check_winner()

        # Case 1: Game is over and there is a winner
        if self.winner is not None:
            self.done = True
            if self.winner == self.current_player:
                return self.board, 1, True, {}  # Big reward for winning
            return self.board, -0.25, True, {} # This case should not be possible (opponent winning on player's turn)
            
        # Case 2: Game is over and there is no winner
        elif np.all(self.board != 0):
            self.done = True
            return self.board, 0, True, {}
        
        # Case 3: Game is still going
        else:
            self.current_player = -self.current_player
            return self.board, 0, False, {}

    def check_winner(self):
        for player in [-1, 1]:
            for i in range(3):
                if np.all(self.board[i, :] == player) or np.all(self.board[:, i] == player):
                    return player
            if np.all(np.diag(self.board) == player) or np.all(np.diag(np.fliplr(self.board)) == player):
                return player
        return None

    def render(self):
        for row in self.board:
            print(" ".join([[" ", "O", "X"][cell] for cell in row]))
        print("\n")
    
    def get_state_string(self):
        return self.to_state_string(self.board)
    
    def to_state_string(self, state):
        state_str = ""
        for row in state:
            state_str += "".join([["-", "O", "X"][cell] for cell in row])
        return state_str

In [121]:
class QLearningParams:
    """Class to encapsulate parameters used for Q-Learning"""
    def __init__(self, alpha: float, gamma: float, epsilon: float):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

In [122]:
class QTable:
    """Simple Q Learning values table implemented using a dictionary"""
    
    def __init__(self, default_q_val: float = 0, actions_range: tuple = (0,9)):
        # Q Table implementation as a dictionary 
        self._table = {}
        self._default = default_q_val
        self._actions_range = actions_range
    
    def get(self, state, action) -> float:
        """Return the Q value for a given state and action"""
        self._register(state)
        return self._table.get(state).get(action)

    def set(self, state, action, val) -> None:
        """Set the Q value for a given state and action"""
        self._register(state)
        self._table[state][action] = val

    def best_action(self, state) -> int:
        """Find the action with the maximum Q value for a given state"""
        self._register(state)
        max_q = None
        best_actions = []
        for action in self._table.get(state, {}).keys():
            q = self._table.get(state).get(action)
            if max_q is None:
                max_q = q
                best_actions.append(action)
            elif max_q > q: 
                continue
            elif max_q == q:
                best_actions.append(action)
            else:
                best_actions = [action]
                max_q = q
        if len(best_actions) == 1:
            return best_actions[0]
        return choice(best_actions)

    def _register(self, state) -> None:
        """Ensure that a given state exists with the Q table. If it does not, add the entry with the default Q value."""
        if self._table.get(state, None) is None:
            self._table[state] = {}
            for action in range(self._actions_range[0], self._actions_range[1]):
                self._table[state][action] = self._default

    def __str__(self):
        return str(self._table)

In [123]:
class AITicAgent:
    """Reinforcement learning agent which uses Q-Learning to determine tic-tac-toe game action"""
    def __init__(self, action_space: gym.spaces.Discrete, q_params: QLearningParams):
        self.action_space = action_space
        self.q_params = q_params
        self.q_table = QTable()
    
    def choose_action(self, state: str):
        """Choose the next action for a given board state using Q-Learning technique"""
        if np.random.uniform(0, 1) < self.q_params.epsilon:
            return self.action_space.sample()
        else:
            return self.q_table.best_action(state)
    
    def update_q_table(self, state: tuple, action: int, reward: int, next_state: tuple):
        """Update Q Table to adjust params for a given state using the best next action """
        # Declare parameters
        best_next_action = self.q_table.best_action(next_state)
        a = self.q_params.alpha
        g = self.q_params.gamma
        current_q = self.q_table.get(state, action)
        next_q = self.q_table.get(next_state, best_next_action)

        # Perform Q Table update calculation
        new_q = (1 - a) * current_q + a * (reward + g * next_q)
        self.q_table.set(state, action, new_q)

In [127]:
# Train two agents with slightly differing configuration
env = TicTacToeEnv()

# Setup the 2 agent players
agent_a = AITicAgent(env.action_space, QLearningParams(0.1, 0.9, 0.1))
agent_b = AITicAgent(env.action_space, QLearningParams(0.075, 0.9, 0.15))
players = [agent_a, agent_b]

# Train for some large number of episodes (games)
# * NOTE: There are 3^9 = 19,683 possible states of the tic-tac-toe board. Using a higher number of training episodes 
# * means greater likelihood an agent understands the value of each possible state.
num_episodes = 50000

for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards = [0, 0]
    player_num = 0

    while not done:
        # Current player takes a turn:
        action = players[player_num].choose_action(env.to_state_string(state))
        next_state, reward, done, _ = env.step(action)
        players[player_num].update_q_table(env.to_state_string(state), action, reward, env.to_state_string(next_state))
        state = next_state
        rewards[player_num] += reward

        # Switch to next player:
        player_num = (player_num-1)*(-1)

    if episode % 1000 == 0:
        print(f"Episode {episode}, A's Total Reward: {rewards[0]}, B's Total Reward: {rewards[1]}")



Episode 0, A's Total Reward: 1, B's Total Reward: 0
Episode 1000, A's Total Reward: 0, B's Total Reward: -1
Episode 2000, A's Total Reward: -2, B's Total Reward: -4
Episode 3000, A's Total Reward: -1, B's Total Reward: 1
Episode 4000, A's Total Reward: 0, B's Total Reward: -1
Episode 5000, A's Total Reward: 1, B's Total Reward: -1
Episode 6000, A's Total Reward: -5, B's Total Reward: -6
Episode 7000, A's Total Reward: -2, B's Total Reward: -2
Episode 8000, A's Total Reward: -1, B's Total Reward: -3
Episode 9000, A's Total Reward: 1, B's Total Reward: 0
Episode 10000, A's Total Reward: 1, B's Total Reward: -3
Episode 11000, A's Total Reward: -3, B's Total Reward: 0
Episode 12000, A's Total Reward: -1, B's Total Reward: 0
Episode 13000, A's Total Reward: 1, B's Total Reward: 0
Episode 14000, A's Total Reward: 1, B's Total Reward: -1
Episode 15000, A's Total Reward: 1, B's Total Reward: -1
Episode 16000, A's Total Reward: 0, B's Total Reward: -1
Episode 17000, A's Total Reward: -1, B's To

In [129]:
# Test a trained agent against an untrained agent:
test_episodes = 100
win_count = 0
agent_c = AITicAgent(env.action_space, QLearningParams(0.1, 0.9, 0.1))
players = [agent_a, agent_c]

for _ in range(test_episodes):
    state = env.reset()
    done = False
    player_num = 0

    while not done:
        action = players[player_num].choose_action(env.get_state_string()) 
        state, _, done, _ = env.step(action)
        player_num = (player_num-1)*(-1)

    if env.winner == 1:  # Trained agent wins (note env stores players differently)
        win_count += 1

# Expect the trained agent to win significant majority of the games
print(f"Trained agent wins {win_count}/{test_episodes} games.")

Trained agent wins 89/100 games.


In [130]:
# Test a trained agent against an untrained agent:
test_episodes = 100
win_count = 0
players = [agent_a, agent_b]

for _ in range(test_episodes):
    state = env.reset()
    done = False
    player_num = 0

    while not done:
        action = players[player_num].choose_action(env.get_state_string()) 
        state, _, done, _ = env.step(action)
        player_num = (player_num-1)*(-1)

    if env.winner == 1:  # Agent A wins (note env stores players differently)
        win_count += 1

# Expect the winrate between the two trained agents to be similar
print(f"Agent A wins {win_count}/{test_episodes} games.")
print(f"Agent B wins {test_episodes - win_count}/{test_episodes} games.")

Agent A wins 57/100 games.
Agent B wins 43/100 games.
